In [1]:
import pytesseract
from PIL import Image,ImageGrab
import cv2 as cv
import threading
import numpy as np

功能：

- get_tesseract_version　　返回系统中安装的Tesseract版本。
- image_to_string　　将图像上的Tesseract OCR运行结果返回到字符串
- image_to_boxes　　返回包含已识别字符及其框边界的结果
- image_to_data　　返回包含框边界，置信度和其他信息的结果。需要- Tesseract 3.05+。有关更多信息，请查看Tesseract TSV文档
- image_to_osd　　返回包含有关方向和脚本检测的信息的结果。

参数：

`image_to_data(image, lang=None, config='', nice=0, output_type=Output.STRING)`

- image object　　图像对象
- lang String，Tesseract　　语言代码字符串
- config String　　任何其他配置为字符串，例如：config='--psm 6'
- output_type　　类属性，指定输出的类型，默认为string。有关所有支持类型的完整列表，请检查pytesseract.Output类的定义。

上层：(500,265,960,305)

下层：(500,400,960,440)

box: (40,15)

bg_color: (2,117,124)

border_color : (100,100,100)

gray: (83,83,83)

改进：

1. 设置参数只识别数字
2. 每次只识别一行，而不是多行

In [3]:
bg_color = (16,24,26)
border_color  = (100,100,100)
def process_img(image):
    pixels = image.load()
    for i in range(image.size[0]):
        for j in range(image.size[1]):
            if pixels[i,j] == bg_color:
                pixels[i,j] = (255,255,255)
                # pass
            

In [4]:
def identify_image(image,lang='eng',show_image=False,config='')->str:
    # process_img(image)
    if show_image:
        image.show()
    return pytesseract.image_to_string(image, lang='eng',config=config)

In [9]:
full_screen = (0,0,1366,768)
config = '--psm 6 -c tessedit_char_whitelist=0123456789.'
def get_parameters(boxes:list):
    screen = ImageGrab.grab(full_screen)
    for box in boxes:
        part_img = screen.crop((box[0],box[1],box[0]+box[2],box[1]+box[3]))
        print(identify_image(image=part_img,show_image=False,config=config))

In [58]:
boxes = [
    (125,395,50,25),
    (125,395+25,50,25),
    (125,395+50,50,25),
    (125,395+75,50,25),
    (125,395+100,50,25),
]
get_parameters(boxes)


50



750

50



In [13]:
image = Image.open('./resource/2.png')
image = image.convert('L')
image.show()
print(pytesseract.image_to_string(image,config=config))


In [14]:
dst = None
def custom_sharpen_demo(image):
  kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], np.float32) #锐化
  dst = cv.filter2D(image, -1, kernel=kernel)
  cv.imshow("custom_blur_demo", dst)
  
src = cv.imread("./resource/1.png")
cv.namedWindow("input image", cv.WINDOW_AUTOSIZE)
cv.imshow("input image", src)
cv.imwrite(dst,'./resource/dist.png')
custom_sharpen_demo(src)
cv.waitKey(0)
cv.destroyAllWindows()

NameError: name 'dst' is not defined

In [135]:
%%time
box_size = (38,15)
v_box_size = (38,36)
offset_x = 60

full_scree = (0,0,1366,768)

up_start_pos1 = (499,266)
up_start_pos2 = (499,287)

down_start_pos1 = (499,401)
down_start_pos2 = (499,422)

start_pos = [up_start_pos1,up_start_pos2,down_start_pos1,down_start_pos2]

def get_parameters(start_pos,box_size):
    count: int = 8
    screen = ImageGrab.grab(full_scree)
    for pos in start_pos:
        for i in range(count):
            new_start = (pos[0]+i*offset_x,pos[1])
            boxx = (new_start[0],new_start[1],new_start[0]+box_size[0],new_start[1]+box_size[1])
            part_img = screen.crop(boxx)

            print(identify_image(part_img))
get_parameters([up_start_pos1,down_start_pos1],(38,36))
# threading.Thread(target=get_parameters,args=([up_start_pos1],v_box_size)).start()

# threading.Thread(target=get_parameters,args=([down_start_pos1],v_box_size)).start()


25
270

25
250

25
225

25
210

25
205

25
185

24
165

24

25
270

25
250

24
225

24
210

25
205

24
185

24
165

24
150

Wall time: 3.51 s
